# 문서 임베딩 생성 및 벡터 데이터베이스 구축

이 노트북에서는 문서 데이터를 임베딩하고 벡터 데이터베이스에 저장하는 과정을 다룹니다.
주요 단계는 다음과 같습니다:

1. JSONL 파일에서 문서 데이터 로드
2. LangChain Document 객체로 변환
3. OpenAI 임베딩 모델을 사용하여 벡터 생성
4. ChromaDB에 벡터 저장
5. 간단한 검색 테스트

시작하기 전에 필요한 라이브러리를 설치해야 합니다:
```
pip install langchain langchain-openai chromadb
```

## 1. 필요한 라이브러리 임포트

In [1]:
import os
import json
from typing import List, Dict, Any
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

## 2. JSONL 파일 로드 함수

JSONL(JSON Lines) 형식의 파일에서 문서 데이터를 로드하는 함수입니다.

In [2]:
def load_jsonl(file_path: str) -> List[Dict[str, Any]]:
    """
    JSONL 파일을 읽어서 문서 리스트로 반환합니다.
    
    Args:
        file_path: JSONL 파일 경로
        
    Returns:
        문서 리스트
    """
    documents = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():  # 빈 줄 무시
                documents.append(json.loads(line))
    return documents

## 3. LangChain Document 객체로 변환

일반 문서 데이터를 LangChain의 Document 객체로 변환하는 함수입니다.

In [3]:
def convert_to_langchain_documents(documents: List[Dict[str, Any]]) -> List[Document]:
    """
    문서 리스트를 LangChain Document 객체 리스트로 변환합니다.
    
    Args:
        documents: 문서 리스트
        
    Returns:
        LangChain Document 객체 리스트
    """
    langchain_docs = []
    for doc in documents:
        langchain_docs.append(
            Document(
                page_content=doc["content"],
                metadata=doc["metadata"]
            )
        )
    return langchain_docs

## 4. OpenAI 임베딩 모델 생성

OpenAI의 임베딩 모델을 초기화하는 함수입니다. 이 함수를 사용하기 위해서는 OpenAI API 키가 환경 변수에 설정되어 있어야 합니다.

In [4]:
def create_embeddings():
    """
    OpenAI 임베딩 모델을 생성합니다.
    
    Returns:
        OpenAI 임베딩 모델
    """
    # OpenAI 임베딩 모델 사용
    # 환경 변수 OPENAI_API_KEY가 설정되어 있어야 합니다
    return OpenAIEmbeddings(model="text-embedding-ada-002")

## 5. OpenAI API 키 설정

아래 셀에서 OpenAI API 키를 설정할 수 있습니다. 보안을 위해 실제 키는 노트북에 저장하지 마세요.

In [5]:
# OpenAI API 키 설정 (실제 키는 노트북에 저장하지 마세요)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# 또는 .env 파일에서 로드할 수도 있습니다
# from dotenv import load_dotenv
# load_dotenv()

## 6. 메인 실행 코드

전체 프로세스를 실행하는 메인 코드입니다.

In [6]:
# 입력 파일 경로
input_file = "../step_01_data_set/split_docs.jsonl"

# 출력 디렉토리 (ChromaDB 저장 위치)
output_dir = "../chroma_db"

# 디렉토리가 없으면 생성
os.makedirs(output_dir, exist_ok=True)

print(f"JSONL 파일 로딩 중: {input_file}")
documents = load_jsonl(input_file)
print(f"총 {len(documents)}개의 문서를 로딩했습니다.")

JSONL 파일 로딩 중: ../step_01_data_set/split_docs.jsonl
총 237개의 문서를 로딩했습니다.


In [7]:
# LangChain Document 객체로 변환
langchain_docs = convert_to_langchain_documents(documents)

# 테스트를 위해 문서 수를 제한할 수 있습니다
# langchain_docs = langchain_docs[:2]

print(f"변환된 문서 수: {len(langchain_docs)}")

# 첫 번째 문서 내용 확인
if langchain_docs:
    print("\n첫 번째 문서 샘플:")
    print(f"내용: {langchain_docs[0].page_content[:150]}...")
    print(f"메타데이터: {langchain_docs[0].metadata}")

변환된 문서 수: 237

첫 번째 문서 샘플:
내용: # Community Projects  
Welcome to the Roo Code community section! Here you'll find community projects that extend Roo Code's capabilities and a galler...
메타데이터: {'path': 'community.md', 'name': 'community.md', 'header1': 'Community Projects'}


## 7. 임베딩 생성 및 ChromaDB에 저장

문서를 임베딩하고 ChromaDB에 저장하는 과정입니다.

In [8]:
# 임베딩 모델 생성
embeddings = create_embeddings()

print("ChromaDB에 문서 추가 중...")
# ChromaDB 벡터 저장소 생성 및 문서 추가
vectordb = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embeddings,
    persist_directory=output_dir
)

# 벡터 저장소 저장
vectordb.persist()
print(f"임베딩 완료. 벡터 데이터베이스가 {output_dir}에 저장되었습니다.")

ChromaDB에 문서 추가 중...
임베딩 완료. 벡터 데이터베이스가 ../chroma_db에 저장되었습니다.


/tmp/ipykernel_2082637/3696109469.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## 8. 벡터 검색 테스트

생성된 벡터 데이터베이스를 사용하여 간단한 검색 테스트를 수행합니다.

In [9]:
# 간단한 검색 테스트
query = "Roo Code란 무엇인가요?"
print(f"\n테스트 쿼리: '{query}'")
results = vectordb.similarity_search(query, k=2)

print("\n검색 결과:")
for i, doc in enumerate(results):
    print(f"\n결과 {i+1}:")
    print(f"내용: {doc.page_content[:150]}...")
    print(f"메타데이터: {doc.metadata}")


테스트 쿼리: 'Roo Code란 무엇인가요?'

검색 결과:

결과 1:
내용: # Roo Code Docs  
Roo Code (formerly Roo Cline) is an AI-powered autonomous coding agent that lives in your editor. It helps you code faster and smart...
메타데이터: {'header1': 'Roo Code Docs', 'name': 'index.md', 'path': 'index.md'}

결과 2:
내용: ## General  
### What is Roo Code?  
Roo Code is an AI-powered autonomous coding agent that lives in your editor.  
### How does Roo Code work?  
Roo ...
메타데이터: {'header1': 'Frequently Asked Questions', 'header2': 'General', 'name': 'faq.md', 'path': 'faq.md'}


## 9. 추가 실험: 다양한 쿼리 테스트

다양한 쿼리로 벡터 검색을 테스트해볼 수 있습니다.

In [10]:
# 다양한 쿼리로 테스트
test_queries = [
    "Roo Code의 주요 기능은 무엇인가요?",
    "Roo Code를 어떻게 설치하나요?",
    "Roo Code의 장점은 무엇인가요?"
]

for query in test_queries:
    print(f"\n\n쿼리: '{query}'")
    results = vectordb.similarity_search(query, k=1)
    
    print("검색 결과:")
    for i, doc in enumerate(results):
        print(f"\n결과 {i+1}:")
        print(f"내용: {doc.page_content[:150]}...")
        print(f"메타데이터: {doc.metadata}")



쿼리: 'Roo Code의 주요 기능은 무엇인가요?'
검색 결과:

결과 1:
내용: ## What Can Roo Code Do?  
- 🚀 **Generate Code** from natural language descriptions
- 🔧 **Refactor & Debug** existing code
- 📝 **Write & Update** docu...
메타데이터: {'header1': 'Roo Code Docs', 'header2': 'What Can Roo Code Do?', 'name': 'index.md', 'path': 'index.md'}


쿼리: 'Roo Code를 어떻게 설치하나요?'
검색 결과:

결과 1:
내용: # Installing Roo Code  
Roo Code is a VS Code extension that brings AI-powered coding assistance to your editor.  There are three main ways to install...
메타데이터: {'header1': 'Installing Roo Code', 'name': 'installing.md', 'path': 'getting-started/installing.md'}


쿼리: 'Roo Code의 장점은 무엇인가요?'
검색 결과:

결과 1:
내용: ## What Can Roo Code Do?  
- 🚀 **Generate Code** from natural language descriptions
- 🔧 **Refactor & Debug** existing code
- 📝 **Write & Update** docu...
메타데이터: {'header1': 'Roo Code Docs', 'header2': 'What Can Roo Code Do?', 'name': 'index.md', 'path': 'index.md'}
